In [2]:
import pygame
import random
from enum import Enum
import time

# 각종 변수 지정
SCREEN_WIDTH = 480  # 스크린 가로 길이
SCREEN_HEIGHT = 680  # 스크린 세로 길이
PLAYER_COL_AREA = (30, 50)  # 플레이어의 충돌 렉트 크기
mp = 10  # 최대 pumpkin 갯수
mc = 10  # 최대 사탕 시간
mb= 1  # 최대 사탕 시간
SCORE = 0
LIFE = 5
candy2_list = []

# 화면 그리기(렌더)를 위한 레이어(0번부터 그려짐)
class RenderLayer(Enum):
    Back = 0
    Middle = 1
    Front = 2
    F2 = 3
    F3 = 4
    Max = 5


# 충돌 객체를 나누기 위한 레이어
class CollisionLayer(Enum):
    ColNone = 0
    GroupA = 1
    GroupB = 2
    GroupC = 3
    GroupD = 4
    Max = 5


# 플레이어의 상태를 저장하기 위한 Enum
class PlayerState(Enum):
    Idle = 0
    Left = 1
    Right = 2
    Dead = 3
    Max = 4


# 부모가 되는 게임오브젝트 
class GameObject:
    def __init__(self, image_list, render_layer, coll_layer, pos_x=0, pos_y=0):
        self.image_list = image_list  # 이미지는 리스트로 받음 기본 이미지는 1번, 0번은 안 그릴 때 사용
        self.rect = self.image_list[1].get_rect()  # 이미지에서 렉트 정보 받아옴
        self.width = self.rect.size[0]  # 오브젝트의 가로 길이
        self.height = self.rect.size[1]  # 오브젝트의 세로 길이
        self.col_width = self.width  # 충돌 렉트의 가로 길이(기본은 이미지와 동일)
        self.col_height = self.height  # 충돌 렉트의 세로 길이(기본은 이미지와 동일)
        self.pos_x = pos_x  # 오브젝트의 x좌표
        self.pos_y = pos_y  # 오브젝트의 y좌표
        self.render_time = 0  # 렌더를 위한 타이머
        object_list.append(self)  # 업데이트 함수 호출을 위해 리스트에 저장
        render_list[render_layer].append(self)  # 각 렌더 레이어 리스트에 저장
        collision_list[coll_layer].append(self)  # 각 충돌 레이어 리스트에 저장

    # 초기화를 위한 함수
    def initialize(self):
        pass

    # 업데이트를 위한 함수
    def update(self):
        pass

    # 렌더(그리기)를 위한 함수. index의 기본값은 1이고 0이면 그리지 않음
    def render(self, index=1):
        if index > 0:
            screen.blit(self.image_list[index], (self.pos_x, self.pos_y))


# 플레이어부터는 게임오브젝트 클래스 상속
class Player(GameObject):
    def __init__(self, image, render_layer, coll_layer, pos_x=0, pos_y=0):
        super().__init__(image, render_layer, coll_layer, pos_x, pos_y)
        self.state = PlayerState.Idle.value  # 플레이어의 상태값

    def initialize(self):
        self.state = PlayerState.Idle.value
        self.pos_x = background.width / 2 - player.width / 2  # 플레이어 위치를 가운데로 지정
        self.pos_y = background.height - platform.height - player.height  # 플레이어 위치를 바닥 위로 설정

    def update(self):
        key = pygame.key.get_pressed()  # 키 상태를 받기 위한 변수
        speed = 700  # 이동 스피드

        # 플레이어가 죽은 상태면 키보드 입력 안받음
        if self.state == PlayerState.Dead.value:
            return

        self.state = PlayerState.Idle.value  # 기본적으로 플레이어 상태를 idle로 설정

        # 왼쪽 키를 누르면 왼쪽으로 이동하고 상태를 Left로 설정
        if key[pygame.K_LEFT] == 1:
            self.pos_x -= speed * clock.get_time() / 1000
            self.state = PlayerState.Left.value

        # 오른쪽 키를 누르면 오른쪽으로 이동하고 상태를 Right로 설정
        if key[pygame.K_RIGHT] == 1:
            self.pos_x += speed * clock.get_time() / 1000
            self.state = PlayerState.Right.value

        # 플레이어의 x좌표가 왼쪽 한계값을 넘지 못하게 제어
        if self.pos_x < 0 - (self.width / 2 - self.col_width / 2):
            self.pos_x = 0 - (self.width / 2 - self.col_width / 2)

        # 플레이어의 x좌표가 오른쪽 한계값을 넘지 못하게 제어
        if self.pos_x > SCREEN_WIDTH - self.width + (self.width / 2 - self.col_width / 2):
            self.pos_x = SCREEN_WIDTH - self.width + (self.width / 2 - self.col_width / 2)

    def render(self, index=1):
        self.render_time += clock.get_time() / 1000  # 애니메이션 재생을 위한 렌더 타이머
        render_index = index  # 애니메이션 재생을 위한 인덱스

        # 플레이어의 상태가 idle이면 1, 2번 이미지 교차 재생
        if self.state == PlayerState.Idle.value:
            render_index = 1
        # 플레이어의 상태가 Left면 3, 4번 이미지 교차 재생
        if self.state == PlayerState.Left.value:
            render_index = 1
        # 플레이어의 상태가 Right이면 5, 6번 이미지 교차 재생
        if self.state == PlayerState.Right.value:
            render_index = 2
        # 플레이어의 상태가 dead면 사망 이미지 재생
        if self.state == PlayerState.Dead.value:
            render_index =3

        # 해당 이미지의 인덱스를 부모의 render함수에 전달
        super().render(render_index)


#호박
class Pumpkin(GameObject):
    def __init__(self, image, render_layer, coll_layer, pos_x=0, pos_y=0):
        super().__init__(image, render_layer, coll_layer, pos_x, pos_y)
        self.speed = 0  # 호박의 속도를 위한 변수
        self.pop = False  # 별의 사라짐 여부를 체크하기 위한 변수

    # 초기화 함수를 호출할 때마다 별의 위치를 재설정
    def initialize(self):
        self.pos_x = random.randint(0, SCREEN_WIDTH - self.width)  # x좌표는 스크린의 width 안쪽에서
        self.pos_y = -random.randint(self.image_list[1].get_size()[1], 300)  # y좌표는 -300쯤에서 랜덤으로 설정
        self.speed = 2
        self.pop = False
        self.render_time = 0

    def update(self):
        # 별의 위치가 0을 넘어서면 점차 가속도가 붙음
        if self.pos_y > 0:
            self.speed += 10 * clock.get_time() / 1000

        # 호박이 화면을 벗어나면 사라짐(실행되지 않음)
        if self.pos_y > SCREEN_HEIGHT:
            self.pop = True

        # 호박이 사라지지 않았을 때만 떨어짐
        if self.pop is False:
            self.pos_y += self.speed

    def render(self, index=1):
        render_index = index

        # 호박이 바닥에 닿으면 삭제
        if self.pop:
            self.initialize()

        # 해당 이미지의 인덱스를 부모의 render함수에 전달
        super().render(render_index)
        
#사탕        
class Candy(GameObject):
    def __init__(self, image, render_layer, coll_layer, pos_x=0, pos_y=0):
        super().__init__(image, render_layer, coll_layer, pos_x, pos_y)
        self.speed = 0  # 사탕의 속도를 위한 변수
        self.pop = False  # 사탕의 사라짐 여부를 체크하기 위한 변수

    # 초기화 함수를 호출할 때마다 사탕의 위치를 재설정
    def initialize(self):
        self.pos_x = random.randint(0, SCREEN_WIDTH - self.width)  # x좌표는 스크린의 width 안쪽에서
        self.pos_y = -random.randint(self.image_list[1].get_size()[1], 300)  # y좌표는 -300쯤에서 랜덤으로 설정
        self.speed = 2
        self.pop = False
        self.render_time = 0

    def update(self):
        # 사탕의 위치가 0을 넘어서면 점차 가속도가 붙음
        if self.pos_y > 0:
            self.speed += 10 * clock.get_time() / 1000

        # 사탕이 화면을 벗어나면 사라짐(실행되지 않음)
        if self.pos_y > SCREEN_HEIGHT:
            self.pop = True

        # 사탕이 사라지지 않았을 때만 떨어짐
        if self.pop is False:
            self.pos_y += self.speed

    def render(self, index=1):
        render_index = index

        # 사탕이 바닥에 닿으면 삭제
        if self.pop:
            self.initialize()

        # 해당 이미지의 인덱스를 부모의 render함수에 전달
        super().render(render_index)
        
#보너스        
class Bonus(GameObject):
    def __init__(self, image, render_layer, coll_layer, pos_x=0, pos_y=0):
        super().__init__(image, render_layer, coll_layer, pos_x, pos_y)
        self.speed = 0  # 보너스의 속도를 위한 변수
        self.pop = False  # 보너스의 사라짐 여부를 체크하기 위한 변수

    # 초기화 함수를 호출할 때마다 사탕의 위치를 재설정
    def initialize(self): 
        self.pos_x = random.randint(0, SCREEN_WIDTH - self.width)  # x좌표는 스크린의 width 안쪽에서
        self.pos_y = -random.randint(self.image_list[1].get_size()[1], 300)  # y좌표는 -300쯤에서 랜덤으로 설정
        self.speed = 2
        self.pop = False
        self.render_time = 0

    def update(self):
        # 보너스의 위치가 0을 넘어서면 점차 가속도가 붙음
        if self.pos_y > 0:
            self.speed += 10 * clock.get_time() / 1000

        # 보너스 화면을 벗어나면 사라짐(실행되지 않음)
        if self.pos_y > SCREEN_HEIGHT:
            self.pop = True

        # 사탕이 사라지지 않았을 때만 떨어짐
        if self.pop is False:
            self.pos_y += self.speed
            
          

    def render(self, index=1):
        render_index = index

        # 보너스가 바닥에 닿으면 삭제
        if self.pop:
            self.initialize()

        # 해당 이미지의 인덱스를 부모의 render함수에 전달
        super().render(render_index)

class Candy2(GameObject):
    def __init__(self, image, render_layer, coll_layer, pos_x=0, pos_y=0):
        super().__init__(image, render_layer, coll_layer, pos_x, pos_y)
        self.speed = 0  # 사탕의 속도를 위한 변수
        self.pop = False  # 사탕의 사라짐 여부를 체크하기 위한 변수

    # 초기화 함수를 호출할 때마다 사탕의 위치를 재설정
    def initialize(self):
        self.pos_x = random.randint(0, SCREEN_WIDTH - self.width)  # x좌표는 스크린의 width 안쪽에서
        self.pos_y = -random.randint(self.image_list[1].get_size()[1], 300)  # y좌표는 -300쯤에서 랜덤으로 설정
        self.speed = 2
        self.pop = False
        self.render_time = 0
        return
        

    def update(self):
        # 사탕의 위치가 0을 넘어서면 점차 가속도가 붙음
        if self.pos_y > 0:
            self.speed += 10 * clock.get_time() / 1000

        # 사탕이 화면을 벗어나면 사라짐(실행되지 않음)
        if self.pos_y > SCREEN_HEIGHT:
            self.pop = True

        # 사탕이 사라지지 않았을 때만 떨어짐
        if self.pop is False:
            self.pos_y += self.speed

    def render(self, index=1):
        render_index = index

        # 사탕이 바닥에 닿으면 삭제
        if self.pop:
            self.initialize()

        # 해당 이미지의 인덱스를 부모의 render함수에 전달
        super().render(render_index)
        



# 보통은 텍스트를 객체로 만들 일이 없지만 특별한 렌더 순서 적용을 위해 만듦
class Text(GameObject):
    def __init__(self, image, render_layer, coll_layer, pos_x=0, pos_y=0):
        super().__init__(image, render_layer, coll_layer, pos_x, pos_y)
        self.running_time = 0

    def initialize(self):
        self.running_time = 0  # 시간을 0으로 초기화

    def update(self):
        # 플레이 시간을 누적한 뒤 렌더(round함수로 반올림 뒤 str함수로 문자열로 만듦)
        self.running_time += clock.get_time() / 1000
        self.image_list[1] = timer_font.render(str(round(self.running_time, 1)), True, (255, 255, 255))


# 충돌 계산을 위해 렉트 객체를 만들어주는 함수
def calc_rect_collider(game_object):
    # 렉트 객체 초기화
    rect = pygame.Rect(0, 0, 0, 0)

    # 오브젝트가 플레이어라면 충돌 렉트 사이즈에 맞게 재조정
    if isinstance(game_object, Player):
        rect.left = game_object.pos_x + game_object.width / 2 - game_object.col_width / 2
        rect.top = game_object.pos_y + game_object.height - game_object.col_height
        rect.width = game_object.col_width
        rect.height = game_object.col_height
    # 나머지 경우에는 이미지 크기와 위치를 그대로 적용
    else:
        rect.left = game_object.pos_x
        rect.top = game_object.pos_y
        rect.width = game_object.col_width
        rect.height = game_object.col_height

    return rect  # 렉트 객체를 반환


# 두 객체가 렉트 충돌을 하는지 판별하는 함수
def check_collide_rect(a, b):
    # calc_rect_collider함수로 렉트 객체를 생성한 뒤 비교
    a.rect = calc_rect_collider(a)
    b.rect = calc_rect_collider(b)

    # 충돌 결과를 리턴
    return a.rect.colliderect(b.rect)


# 파이게임을 초기화하고 스크린 크기를 지정
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

# 윈도우 타이틀을 설정
pygame.display.set_caption('Halloween Game!!')

# # 아이콘 이미지 로드 및 적용
# icon_image = pygame.image.load('resources/images/icon.png')
# pygame.display.set_icon(icon_image)

# # 배경음악 로드 및 적용
# pygame.mixer.music.load('resources/sounds/bgm.mp3')
# pygame.mixer.music.set_volume(0.4)  # 볼륨 적용
# pygame.mixer.music.play(-1)  # 반복할 숫자를 인자로 넣음. -1은 반복재생

# # 별이 깨지는 효과음 로드 및 적용
# pop_sound = pygame.mixer.Sound('resources/sounds/coin.wav')
# pop_sound.set_volume(0.7)

# 이미지 객체 생성, 리스트에 None을 첫 번째 요소로 두어 render함수에 인자로 0이 들어왔을 때는 그리지 않게 함
background_images = [None, pygame.image.load('gameimages/background.jpg')]  # 배경 이미지
platform_images = [None, pygame.image.load('gameimages/platform.png')]  # 바닥 이미지
# 플레이어 이미지는 1번부터 8번까지 로드
player_images = [None]
for i in range(1,4):
    player_images.append(pygame.image.load('gameimages/ghost' + str(i) + '.png'))
# 호박 이미지는 1번부터 5번까지 로드
pumpkin_images = [None]
pumpkin_images.append(pygame.image.load('gameimages/pumpkin.png'))

#사탕 이미지는 1번부터 5번까지 로드
candy_images = [None]
candy_images.append(pygame.image.load('gameimages/candy1.png'))
    
bonus_image = [None]
bonus_image.append(pygame.image.load('gameimages/bonus.png'))

candy_images2 = [None]
candy_images2.append(pygame.image.load('gameimages/candy2.png'))

# 3가지의 폰트 로드
default_font = pygame.font.Font('나눔손글씨 희망누리.ttf',30) #('resources/fonts/Cafe24SsurroundAir.ttf', 30)
large_font = pygame.font.Font('나눔손글씨 장미체.ttf',60) #('resources/fonts/Cafe24SsurroundAir.ttf', 60)
timer_font = pygame.font.Font('나눔손글씨 백의의 천사.ttf',30) #('resources/fonts/LAB디지털.ttf', 30)

# 시간 객체 생성
clock = pygame.time.Clock()

# 오브젝트 업데이트를 일괄적으로 수행하기 위한 리스트(모든 오브젝트를 이 리스트에 담음)
object_list = []

# 렌더 레이어 enum값을 활용해 5개의 리스트가 들어있는 리스트를 만듦
render_list = []
for i in range(RenderLayer.Max.value):
    render_list.append([])

# 콜리즌 레이어 enum값을 활용해 5개의 리스트가 들어있는 리스트를 만듦
collision_list = []
for i in range(CollisionLayer.Max.value):
    collision_list.append([])

# 배경 객체 생성(맨 뒤 렌더, 충돌 없음)
background = GameObject(background_images,
                        RenderLayer.Back.value,
                        CollisionLayer.ColNone.value)

# 바닥 객체 생성(가운데 렌더, 충돌 그룹 B)
platform = GameObject(platform_images,
                      RenderLayer.Middle.value,
                      CollisionLayer.GroupB.value)
platform.pos_y = background.height - platform.height  # 바닥의 높이 재설정

# 플레이어 객체 생성(가운데 렌더, 충돌 그룹 B)
player = Player(player_images,
                RenderLayer.Middle.value,
                CollisionLayer.GroupB.value)
# 플레이어의 충돌 렉트를 지정한 값으로 설정
player.col_width = PLAYER_COL_AREA[0]
player.col_height = PLAYER_COL_AREA[1]
# 플레이어가 화면 중앙에 오도록 설정
player.pos_x = background.width / 2 - player.width / 2
player.pos_y = background.height - platform.height - player.height

# 호박을 담을 리스트를 만들고 10개의 호박을 미리 생성 - 오브젝트 풀링 기법)
pumpkin_list = []
for i in range(mp):
    # 호박 객체 생성 후 삽입(맨 앞 렌더, 충돌 그룹 A, 위치는 화면에 안 보이게)
    pumpkin_list.append(Pumpkin(pumpkin_images,
                          RenderLayer.F2.value,
                          CollisionLayer.GroupA.value,
                          -100, -100))
    
# 사탕을 담을 리스트를 만들고 10개의 사탕을 미리 생성 - 오브젝트 풀링 기법)
candy_list = []
for i in range(mc):
    # 사탕 객체 생성 후 삽입(맨 앞 렌더, 충돌 그룹 C, 위치는 화면에 안 보이게)
    candy_list.append(Candy(candy_images,
                          RenderLayer.Front.value,
                          CollisionLayer.GroupC.value,
                          100, 100))

# 보너스를 담을 리스트를 만들고 생성 - 오브젝트 풀링 기법)
bonus_list = []
for i in range(mb):
    # 보너스 객체 생성 후 삽입(맨 앞 렌더, 충돌 그룹 D, 위치는 화면에 안 보이게)
    bonus_list.append(Bonus(bonus_image,
                          RenderLayer.F2.value,
                          CollisionLayer.GroupD.value,
                          100, 100))


# 타이머 이미지 생성(가운데 렌더, 충돌 없음)
timer_image = timer_font.render(str(round(0, 1)), True, (255, 255, 255))
game_timer = Text([None, timer_image],  # 이미지는 항상 첫 요소가 None인 리스트가 되어야 하기 때문에 리스트로 처리
                  RenderLayer.F3.value,
                  CollisionLayer.ColNone.value,
                  300, 185)



scene_index = 0  # 씬 관리를 위한 인덱스
is_running = True  # 루프 관리를 위한 변수
init_menu = False  # 초기화를 한 번만 하기 위한 변수
render_gameover_text = False  # 게임오버 텍스트가 한 번만 그려지게 하기 위한 처리
debug_toggle = False  # 디버그 모드를 체크하기 위한 변구

# 게임 루프 시작
while is_running:
    # FPS 설정
    clock.tick(60)

    # 이벤트 체크
    for event in pygame.event.get():
        # 창의 X버튼을 누르면 종료
        if event.type == pygame.QUIT:
            is_running = False
        # 만약 키를 눌렀다면 진입
        if event.type == pygame.KEYDOWN:
            # 현재 메뉴 씬이고 D를 누르면 게임 시작
            if scene_index == 0 and event.key == pygame.K_d:
                scene_index = 1
            # 현재 게임 중이고 F 키를 누르면 디버그 모드 켜거나 끔
            if scene_index == 1 and event.key == pygame.K_f:
                if debug_toggle:
                    debug_toggle = False
                else:
                    debug_toggle = True
            # 현재 게임 종료 씬이고 D를 누르면 메뉴로
            if scene_index == 2 and event.key == pygame.K_d:
                init_menu = False  # 메뉴 초기화 상태 False
                render_gameover_text = False  # 게임오버 텍스트 출력 상태 False
                scene_index = 0

    # 메뉴 씬이라면
    if scene_index == 0:
        # init_menu가 False인 상태에서 모든 오브젝트를 초기화한 후 init_menu True
        if init_menu is False:
            for game_object in object_list:
                game_object.initialize()
            init_menu = True

        # 배경과 바닥을 그림
        screen.blit(background.image_list[1], (background.pos_x, background.pos_y))
        screen.blit(platform.image_list[1], (platform.pos_x, platform.pos_y))

        # 제목 텍스트 표시
        title_text = large_font.render('할로윈 게임!', True, (255, 255, 255))
        text_rect = title_text.get_rect()  # 렉트 정보를 받아옴
        text_rect.centerx = round(SCREEN_WIDTH / 2)  # x좌표를 스크린의 가운데로 설정
        text_rect.y = 300  # y좌표를 설정
        screen.blit(title_text, text_rect)  # 텍스트 렌더

        # 정보 텍스트 표시 1
        start_text = default_font.render('시작 : D 키', True, (255, 45, 150))
        text_rect = start_text.get_rect()
        text_rect.centerx = round(SCREEN_WIDTH / 2)
        text_rect.y = 450
        screen.blit(start_text, text_rect)

        # 정보 텍스트 표시 2
        info_text = default_font.render('호박을 피해 사탕을 먹으세요', True, (255, 45, 150))
        text_rect = info_text.get_rect()
        text_rect.centerx = round(SCREEN_WIDTH / 2)
        text_rect.y = 500
        screen.blit(info_text, text_rect)

        # 정보 텍스트 표시 3
        info2_text = default_font.render('황금사탕은 특수한 효과가 있습니다.', True, (255, 45, 150))
        text_rect = info2_text.get_rect()
        text_rect.centerx = round(SCREEN_WIDTH / 2)
        text_rect.y = 550
        screen.blit(info2_text, text_rect)

    # 게임 씬이라면
    if scene_index == 1:
        # 모든 오브젝트를 계속 업데이트
        for game_object in object_list:
            game_object.update()
        
        # 그룹 A의 호박들과 그룹 B의 플레이어, 바닥을 충돌 체크
        for pumpkin in collision_list[CollisionLayer.GroupA.value]:
            for game_object in collision_list[CollisionLayer.GroupB.value]:
                # 사라지지 않은 별들만 충돌 체크
                if pumpkin.pop is False and check_collide_rect(pumpkin, game_object):
                    # 충돌을 하면 별을 사라지게 함
                    pumpkin.pop = True
                    # 만약 충돌을 한 게 플레이어라면 진입
                    if isinstance(game_object, Player):
                        LIFE -= 1
                        if LIFE ==0:
                            bc = 0
                            game_object.state = PlayerState.Dead.value
                            scene_index = 2

        # 그룹 C의 사탕들과 그룹 B의 플레이어, 바닥을 충돌 체크
        for candy in collision_list[CollisionLayer.GroupC.value]:
            for game_object in collision_list[CollisionLayer.GroupB.value]:
                # 사라지지 않은 별들만 충돌 체크
                if candy.pop is False and check_collide_rect(candy, game_object):
                    # 충돌을 하면 별을 사라지게 함
                    candy.pop = True
                    # 만약 충돌을 한 게 플레이어라면 진입
                    if isinstance(game_object, Player):
                        SCORE += 1
                        
        # 그룹 D의 보너스와 그룹 B의 플레이어, 바닥을 충돌 체크
        for bonus in collision_list[CollisionLayer.GroupD.value]:
            for game_object in collision_list[CollisionLayer.GroupB.value]:
                # 사라지지 않은 보너스들만 충돌 체크
                if bonus.pop is False and check_collide_rect(bonus, game_object):
                    # 충돌을 하면 별을 사라지게 함
                    bonus.pop = True
                    # 만약 충돌을 한 게 플레이어라면 진입
                    if isinstance(game_object, Player):
                       
                            
                       # 보너스사탕을 담을 리스트를 만들고 2개의 사탕을 미리 생성 - 오브젝트 풀링 기법)
                        candy2_list = []
                        for i in range(20):
                            pygame.time.delay(10)
                            # 사탕 객체 생성 후 삽입(맨 앞 렌더, 충돌 그룹 C, 위치는 화면에 안 보이게)
                            candy2_list.append(Candy2(candy_images2,
                                                    RenderLayer.Front.value,
                                                    CollisionLayer.GroupC.value,
                                                    100, 100))
                        
 
        # 렌더 리스트에 있는 오브젝트들은 레이어 순서대로 렌더
        for sub_list in render_list:
            for game_object in sub_list:
                game_object.render()


    # 게임 종료 씬이라면
    if scene_index == 2:
        # render_gameover_text변수를 통해 한 번만 그림
        if render_gameover_text is False:
            # 게임 오버 텍스트 표시
            gameover_text = large_font.render('Game Over', True, (255, 0, 0))
            text_rect = gameover_text.get_rect()
            text_rect.centerx = round(SCREEN_WIDTH / 2)
            text_rect.y = 300
            screen.blit(gameover_text, text_rect)
            
            #점수 텍스트 표시
            score_text = large_font.render('점수 = {}점'.format(SCORE), True, (255, 100, 255))
            text_rect = gameover_text.get_rect()
            text_rect.centerx = round(SCREEN_WIDTH / 2)
            text_rect.y = 375
            screen.blit(score_text, text_rect)

            # 정보 텍스트 표시
            restart_text = default_font.render('메뉴로 : D 키', True, (255, 45, 150))
            text_rect = restart_text.get_rect()
            text_rect.centerx = round(SCREEN_WIDTH / 2)
            text_rect.y = 450
            screen.blit(restart_text, text_rect)
            
            #점수 초기화
            SCORE = 0
            #목숨 초기화
            LIFE = 5
            
            # 변수 True
            render_gameover_text = True

    # 디스플레이 업데이트(필수)
    pygame.display.update()

# while문을 탈출하면 게임 종료
pygame.quit()